![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [4]:
cursor.execute("SELECT * FROM airports")

#### Use description

In [5]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [6]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [34]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print('1. airlines table\n')
print('1a Description\n',cursor.description)
print('\n1b PRAGMA\n',*info, sep = "\n") 

print('-' * 35)
print('\n2. routes table\n')
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print('2a Description\n',cursor.description)
print('\n2b ', *info, sep = "\n") 

1. airlines table

1a Description
 (('cid', None, None, None, None, None, None), ('name', None, None, None, None, None, None), ('type', None, None, None, None, None, None), ('notnull', None, None, None, None, None, None), ('dflt_value', None, None, None, None, None, None), ('pk', None, None, None, None, None, None))

1b PRAGMA

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)
-----------------------------------

2. routes table

2a Description
 (('cid', None, None, None, None, None, None), ('name', None, None, None, None, None, None), ('type', None, None, None, None, None, None), ('notnull', None, None, None, None, None, None), ('dflt_value', None, None, None, None, None, None), ('pk', None, None, None, None, None, None))

2b 
(0, 'index',

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

In [42]:
cursor.execute('''select * from airlines where (country = 'United Kingdom' and active = 'Y');''').fetchall()

[(111,
  '112',
  'Astraeus',
  '\\N',
  '5W',
  'AEU',
  'FLYSTAR',
  'United Kingdom',
  'Y'),
 (491,
  '492',
  'Air Southwest',
  '\\N',
  None,
  'WOW',
  'SWALLOW',
  'United Kingdom',
  'Y'),
 (507,
  '508',
  'Aurigny Air Services',
  '\\N',
  'GR',
  'AUR',
  'AYLINE',
  'United Kingdom',
  'Y'),
 (564,
  '565',
  'Air Wales',
  '\\N',
  '6G',
  'AWW',
  'RED DRAGON',
  'United Kingdom',
  'Y'),
 (664,
  '665',
  'AD Aviation',
  '\\N',
  None,
  'VUE',
  'FLIGHTVUE',
  'United Kingdom',
  'Y'),
 (689, '690', 'Air Foyle', '\\N', 'GS', 'UPA', 'FOYLE', 'United Kingdom', 'Y'),
 (1353,
  '1355',
  'British Airways',
  '\\N',
  'BA',
  'BAW',
  'SPEEDBIRD',
  'United Kingdom',
  'Y'),
 (1409,
  '1411',
  'British International Helicopters',
  '\\N',
  'BS',
  'BIH',
  'BRINTEL',
  'United Kingdom',
  'Y'),
 (1435,
  '1437',
  'bmi',
  'bmi British Midland',
  'BD',
  'BMA',
  'MIDLAND',
  'United Kingdom',
  'Y'),
 (1439, '1441', 'bmibaby', '\\N', 'WW', 'BMI', 'BABY', 'United Kingd

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [ ]:
cursor.execute('''select * from airlines where (country = 'United Kingdom' and active = 'Y');''').fetchall()

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [49]:
cursor.execute('select routes.*, latitude, longitude from airports join routes on airports.id = routes.source_id').fetchall()

[(17312,
  'CG',
  '1308',
  'GKA',
  '1',
  'HGU',
  '3',
  None,
  '0',
  'DH8 DHT',
  '-6.081689',
  '145.391881'),
 (17313,
  'CG',
  '1308',
  'GKA',
  '1',
  'LAE',
  '4',
  None,
  '0',
  'DH8',
  '-6.081689',
  '145.391881'),
 (17314,
  'CG',
  '1308',
  'GKA',
  '1',
  'MAG',
  '2',
  None,
  '0',
  'DH8',
  '-6.081689',
  '145.391881'),
 (17315,
  'CG',
  '1308',
  'GKA',
  '1',
  'POM',
  '5',
  None,
  '0',
  'DH8',
  '-6.081689',
  '145.391881'),
 (46428,
  'PX',
  '328',
  'GKA',
  '1',
  'POM',
  '5',
  None,
  '0',
  'DH4 DH8 DH3',
  '-6.081689',
  '145.391881'),
 (17343,
  'CG',
  '1308',
  'MAG',
  '2',
  'GKA',
  '1',
  None,
  '0',
  'DH8',
  '-5.207083',
  '145.7887'),
 (17344,
  'CG',
  '1308',
  'MAG',
  '2',
  'HGU',
  '3',
  None,
  '0',
  'DH8',
  '-5.207083',
  '145.7887'),
 (17345,
  'CG',
  '1308',
  'MAG',
  '2',
  'LAE',
  '4',
  None,
  '0',
  'DH8',
  '-5.207083',
  '145.7887'),
 (17346,
  'CG',
  '1308',
  'MAG',
  '2',
  'WWK',
  '6',
  None,
  '0',
 

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [2]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [3]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
345,A443876,Max,05/06/2019 04:56:00 PM,05/06/2019 04:56:00 PM,03/21/2006,Adoption,NaN,Cat,Neutered Male,13 years,Domestic Shorthair Mix,Blue Tabby/White
669,A793215,Max,04/30/2019 11:49:00 AM,04/30/2019 11:49:00 AM,04/21/2017,Return to Owner,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Black/White
819,A731802,Max,04/26/2019 06:17:00 PM,04/26/2019 06:17:00 PM,07/27/2015,Adoption,NaN,Dog,Neutered Male,3 years,Maltese Mix,White
820,A789320,Max,04/26/2019 06:01:00 PM,04/26/2019 06:01:00 PM,08/19/2017,Adoption,NaN,Dog,Neutered Male,1 year,Pit Bull Mix,White/Black
1158,A793073,Max,04/19/2019 01:55:00 PM,04/19/2019 01:55:00 PM,04/19/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Boxer Mix,Brown/Tricolor


In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [ ]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", conn)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

## Reflection